In [1]:
import aliases

In [2]:
from everest.ptolemaic.sprite import Sprite

In [3]:
class MySprite(Sprite):

    a : Sprite.Param.Pos()
    b : Sprite.Param.Pos() = 'B'
    c : Sprite.Param.PosKw() = 'C'
    d : Sprite.Param.PosKw() = 'D'
    e : Sprite.Param.Kw()
    f : Sprite.Param.Kw() = 'F'

#     @classmethod
#     def parameterise(cls, register):
#         rpos, opos, rposkw, oposkw, rkw, poskw = cls._classbodyparams
#         for i in range(sum(map(len, (rpos, opos, rposkw, oposkw))))
#         if missing := rpos[register.nargs:]:
#             raise ValueError(
#                 f"Failed to provide required positional arguments:"
#                 f"{set(missing)}"
#                 )
#         register(opos[register.nargs - len(rpos):])
#         register.nargs - rpos
#         register(a, b, c=3)
#         super().parameterise(register)

In [4]:
myobj = MySprite(1, e=3)

In [5]:
myobj

MySprite(a=1, b=B, c=C, d=D, e=3, f=F)